In [12]:
from jupyter_client import find_connection_file
connection_file = find_connection_file()
print(connection_file)

/home/yonatan/.local/share/jupyter/runtime/kernel-v2-168731RGMx4QOLSQ2.json


In [13]:
import papermill as pm
import pandas as pd
import os
import sys
import shutil

sys.tracebacklimit = 0
def exception_handler(exception_type, exception, traceback):
    # All your trace are belong to us!
    # your format
    print(f"{exception_type.__name__}, {exception}")

sys.excepthook = exception_handler

from constants import *
shutil.copy('constants.py', 'export/')

'export/constants.py'

In [14]:
# Read in Data and drop missing values
df = pd.read_csv("../data/all_vaccines.csv")
df.dropna(inplace=True, subset=[immage_col, dataset_col, day_col, response_col])
dataset_names = df["Dataset"].unique().astype(str)

# Plot distribution of N values
N_vals = df[[dataset_col, uid_col]].groupby(dataset_col, as_index=False)[uid_col].nunique()
N_vals = N_vals.rename(columns={uid_col: "N"})

# Narrow to large datasets only
bNarrow = False
N_vals = N_vals.loc[N_vals["N"] > 70]
if bNarrow:
    filtered_df = df.loc[df["Dataset"].isin(N_vals["Dataset"])]
    dataset_names = filtered_df["Dataset"].unique().astype(str)

In [15]:
datasets = pd.DataFrame(dataset_day_dicts)
dataset_names = datasets[dataset_col].unique().astype(str)
filtered_df = df.loc[df["Dataset"].isin(dataset_names)]

In [16]:
exclude_datasets = ["GSE45735.SDY224", "GSE47353.SDY80"] # , "GSE48023.SDY1276", "SDY296"]
bInfluenza = True
bAdjustMFC = False
bDiscardSeroprotected = True
bOlderOnly = True
age_restrict_str = f"_older-only" if bOlderOnly else ""
seroprotected_str = f"_discard_seroprotected" if bDiscardSeroprotected else ""

In [17]:
if bInfluenza:
    datasets = pd.DataFrame(influenza_dicts)
    dataset_names = datasets["Dataset"].unique().astype(str)
    dataset_names = list(set(dataset_names) - set(exclude_datasets))
    
    datasets = datasets.loc[datasets["Dataset"].isin(dataset_names)]
    filtered_df = df.loc[df["Dataset"].isin(dataset_names)]

parameters = {
            "bAdjustMFC" : bAdjustMFC,
            "bDiscardSeroprotected" : bDiscardSeroprotected,
            "bInfluenza": bInfluenza,
            "bOlderOnly": bOlderOnly,
            }

In [18]:
dataset_names = dataset_names[:1]
dataset_names

['SDY67']

In [10]:
# Loop through each combination of dataset and strain
if bAdjustMFC == False:
    for dataset_name in dataset_names:
            dataset = datasets.loc[datasets[dataset_col] == dataset_name]
            dataset_df = filtered_df.loc[filtered_df[dataset_col] == dataset_name]
            print(dataset_name)
            days = dataset["Days"].iloc[0]
            for day in days:
                    print(day)
                    day_mask = dataset_df[day_col] == day
                    name_mask = dataset_df[dataset_col] == dataset_name
                    data = dataset_df.loc[(name_mask) & (day_mask)].reset_index()
                    strains = data[strain_col].unique()
                    print(strains)
                    for strain_index in range(len(strains)):
                            strain_name = strains[strain_index].replace("/", "_").replace(" ", "_")
                            print(f'exporting {dataset_name}, strain no. {strain_index}: {strain_name}, day: {day}')
                            # Define parameters for dataset and strain
                            parameters.update({
                                "dataset_name": dataset_name,
                                "strain_index": strain_index,
                                "day": day
                            })
                            # EXECUTE the notebook with specific parameters
                            output_notebook_name = f"{dataset_name}_{strain_name}_{day}{seroprotected_str}{age_restrict_str}"
                            output_notebook = f"export/{output_notebook_name}.ipynb"
                            try:
                                    pm.execute_notebook(
                                            input_path="vaccines-4.ipynb",
                                            output_path=output_notebook,
                                            parameters=parameters,
                                            prepare_only=True
                                    )
                            except:
                                    print (f"******\nCaught exception when runnnig {output_notebook}\n******\n")
                            # Export the executed notebook to HTML
                            output_html = f"{output_notebook_name}.html"
                            os.system(f"jupyter nbconvert --execute --no-input --to html {output_notebook} --output {output_html}")

SDY67
FC.D28.HAI
['A/Perth/16/2009' 'A/California/7/2009']
exporting SDY67, strain no. 0: A_Perth_16_2009, day: FC.D28.HAI


[NbConvertApp] Converting notebook export/SDY67_A_Perth_16_2009_FC.D28.HAI_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 735329 bytes to export/SDY67_A_Perth_16_2009_FC.D28.HAI_discard_seroprotected.html


exporting SDY67, strain no. 1: A_California_7_2009, day: FC.D28.HAI


[NbConvertApp] Converting notebook export/SDY67_A_California_7_2009_FC.D28.HAI_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 745733 bytes to export/SDY67_A_California_7_2009_FC.D28.HAI_discard_seroprotected.html


HAI.D28
['A/Perth/16/2009' 'A/California/7/2009']
exporting SDY67, strain no. 0: A_Perth_16_2009, day: HAI.D28


...

[NbConvertApp] Converting notebook export/GSE59654.SDY404_A_Perth_16_2009_FC.HAI_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 758391 bytes to export/GSE59654.SDY404_A_Perth_16_2009_FC.HAI_discard_seroprotected.html


exporting GSE59654.SDY404, strain no. 1: A_California_7_2009, day: FC.HAI


[NbConvertApp] Converting notebook export/GSE59654.SDY404_A_California_7_2009_FC.HAI_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 754475 bytes to export/GSE59654.SDY404_A_California_7_2009_FC.HAI_discard_seroprotected.html


exporting GSE59654.SDY404, strain no. 2: B_Brisbane_60_2008, day: FC.HAI


[NbConvertApp] Converting notebook export/GSE59654.SDY404_B_Brisbane_60_2008_FC.HAI_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 749410 bytes to export/GSE59654.SDY404_B_Brisbane_60_2008_FC.HAI_discard_seroprotected.html


In [11]:
if bAdjustMFC == True:
    for dataset_name in dataset_names:
            dataset = datasets.loc[datasets[dataset_col] == dataset_name]
            filtered_df = filtered_df.loc[filtered_df[dataset_col] == dataset_name]
            print(dataset_name)
            strain_name = "Influenza"
            print(f'exporting {dataset_name}, using adjusted MFC')
            # Define parameters for dataset and strain
            parameters.update({
                "day0": dataset["Day0"].iloc[0],
                "dayMFC": dataset["DayMFC"].iloc[0]
            })

            # EXECUTE the notebook with specific parameters
            day = 'Adjusted_MFC'
            output_notebook_name = f"{dataset_name}_{strain_name}_{day}{seroprotected_str}{age_restrict_str}"
            output_notebook = f"export/{output_notebook_name}.ipynb"
            try:
                pm.execute_notebook(
                        input_path="vaccines-4.ipynb",
                        output_path=output_notebook,
                        parameters=parameters,
                        prepare_only=True
                )
            except Exception as e:
                print (f"******\nCaught exception when runnnig {output_notebook}\n******\n")
                print(e)
            # Export the executed notebook to HTML
            output_html = f"{output_notebook_name}.html"
            os.system(f"jupyter nbconvert --execute --no-input --to html {output_notebook} --output {output_html}")